# 33-多线程与多进程

## 📚 用途说明

**学习目标**：
- 掌握Python并发编程的基本概念和原理
- 熟练使用concurrent.futures进行线程和进程管理
- 理解GIL对Python并发性能的影响
- 能够选择合适的并发策略解决实际问题

**前置要求**：
- 已完成32-网络请求处理学习
- 熟练掌握Python函数和面向对象编程
- 了解基本的操作系统概念（进程、线程）

**与LangChain关联**：
- 并发处理是LangChain批量API调用的基础
- 线程池模式影响LangChain的并行处理能力
- I/O密集型任务优化对LangChain性能至关重要
- 为后续学习LangChain的异步处理和流式响应做准备

---

## 🔢 知识点覆盖

### 4.6 多线程与多进程 [⭐⭐进阶]
**知识点说明**：多线程与多进程是Python并发编程的核心技术，能够显著提升I/O密集型和CPU密集型任务的性能。concurrent.futures提供了现代化的并发接口，是实际项目中推荐使用的方案。

**学习要求**：
- 掌握线程和进程的基本概念和区别
- 熟练使用concurrent.futures进行并发编程
- 理解GIL对Python并发性能的影响
- 能够选择合适的并发策略

**案例要求**：
- 实现完整的并发应用示例
- 进行线程和进程性能对比练习
- 应用并发编程解决实际问题
- 验证点：能独立构建高效的并发程序

In [ ]:
print("🔄 并发编程基础概念:")
print("=" * 50)

import threading
import multiprocessing
import concurrent.futures
import time
import os
import queue
from typing import List, Dict, Any, Callable, Optional
from dataclasses import dataclass
from enum import Enum
import random
import hashlib
import json
from datetime import datetime

# 1. 并发编程基本概念
print(f"📝 1. 并发编程基本概念:")

# 1.1 进程与线程的区别
print(f"\n   🔍 1.1 进程与线程的区别:")

@dataclass
class ConcurrencyConcept:
    """并发概念对比"""
    aspect: str
    process_desc: str
    thread_desc: str

concepts = [
    ConcurrencyConcept("内存空间", "独立内存空间，数据不共享", "共享进程内存空间，数据可直接共享"),
    ConcurrencyConcept("创建开销", "开销大，需要复制整个进程", "开销小，创建速度快"),
    ConcurrencyConcept("通信方式", "IPC（管道、队列、共享内存）", "直接共享变量、队列、锁"),
    ConcurrencyConcept("稳定性", "一个进程崩溃不影响其他进程", "一个线程崩溃可能导致整个进程崩溃"),
    ConcurrencyConcept("适用场景", "CPU密集型任务", "I/O密集型任务")
]

print(f"   进程与线程对比:")
for concept in concepts:
    print(f"\n     {concept.aspect}:")
    print(f"       进程: {concept.process_desc}")
    print(f"       线程: {concept.thread_desc}")

# 1.2 GIL（全局解释器锁）
print(f"\n   🔒 1.2 GIL（全局解释器锁）:")

def demonstrate_gil():
    """演示GIL对多线程的影响"""
    def cpu_bound_task(n):
        """CPU密集型任务"""
        while n > 0:
            n -= 1
        return n
    
    def io_bound_task():
        """I/O密集型任务"""
        time.sleep(0.1)  # 模拟I/O等待
        return "I/O完成"
    
    print(f"   GIL说明:")
    print(f"   - Python的GIL确保同一时间只有一个线程执行Python字节码")
    print(f"   - 对于CPU密集型任务，多线程不能真正并行")
    print(f"   - 对于I/O密集型任务，多线程可以有效提升性能")
    print(f"   - 多进程可以绕过GIL限制，实现真正的并行")
    
    # 模拟演示
    print(f"\n   任务类型示例:")
    print(f"   CPU密集型: 大量数学计算、数据压缩、图像处理")
    print(f"   I/O密集型: 网络请求、文件读写、数据库操作")

demonstrate_gil()

# 1.3 并发vs并行
print(f"\n   ⚡ 1.3 并发vs并行:")

class ConcurrencyType(Enum):
    """并发类型"""
    CONCURRENT = "并发"  # 交替执行，看起来同时
    PARALLEL = "并行"   # 真正同时执行

print(f"   并发 (Concurrency):")
print(f"   - 多个任务在同一时间段内交替执行")
print(f"   - 单核CPU上的多线程就是并发")
print(f"   - 适用于I/O密集型任务")

print(f"\n   并行 (Parallelism):")
print(f"   - 多个任务在同一时刻真正同时执行")
print(f"   - 需要多核CPU支持")
print(f"   - 多进程可以实现真正的并行")

# 2. concurrent.futures基础
print(f"\n📝 2. concurrent.futures基础:")

# 2.1 ThreadPoolExecutor
print(f"\n   🧵 2.1 ThreadPoolExecutor:")

def demonstrate_thread_pool():
    """演示线程池的使用"""
    def simple_task(task_id: int) -> str:
        """简单任务"""
        time.sleep(0.1)  # 模拟I/O操作
        return f"任务{task_id}完成"
    
    print(f"   线程池基础使用:")
    
    # 创建线程池
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        print(f"   创建线程池，最大工作线程数: 3")
        
        # 提交任务
        future1 = executor.submit(simple_task, 1)
        future2 = executor.submit(simple_task, 2)
        future3 = executor.submit(simple_task, 3)
        
        print(f"   已提交3个任务")
        
        # 获取结果
        results = []
        for future in concurrent.futures.as_completed([future1, future2, future3]):
            result = future.result()
            results.append(result)
            print(f"   获取结果: {result}")
        
        print(f"\n   所有任务完成，结果数量: {len(results)}")

demonstrate_thread_pool()

# 2.2 ProcessPoolExecutor
print(f"\n   🔄 2.2 ProcessPoolExecutor:")

def demonstrate_process_pool():
    """演示进程池的使用"""
    def cpu_task(n: int) -> int:
        """CPU密集型任务"""
        # 模拟计算密集型操作
        result = sum(i * i for i in range(n))
        return result
    
    print(f"   进程池基础使用:")
    
    # 创建进程池
    with concurrent.futures.ProcessPoolExecutor(max_workers=2) as executor:
        print(f"   创建进程池，最大工作进程数: 2")
        
        # 提交任务
        numbers = [1000, 2000, 3000]
        futures = [executor.submit(cpu_task, n) for n in numbers]
        
        print(f"   已提交{len(futures)}个CPU密集型任务")
        
        # 获取结果
        for i, future in enumerate(concurrent.futures.as_completed(futures)):
            result = future.result()
            print(f"   任务{i+1}结果: {result}")

demonstrate_process_pool()

# 2.3 map函数的使用
print(f"\n   🗺️ 2.3 map函数的使用:")

def demonstrate_map_function():
    """演示map函数的使用"""
    def process_data(data: str) -> str:
        """处理数据"""
        time.sleep(0.05)  # 模拟处理时间
        return f"处理_{data}"
    
    data_list = ["数据1", "数据2", "数据3", "数据4", "数据5"]
    
    print(f"   使用map函数批量处理:")
    print(f"   输入数据: {data_list}")
    
    # 线程池map
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        results = list(executor.map(process_data, data_list))
        
    print(f"   处理结果: {results}")
    print(f"   结果数量: {len(results)}")

demonstrate_map_function()

print(f"\n✅ 并发编程基础概念完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握线程和进程的基本概念和区别")
print(f"   ✓ 理解GIL对Python并发性能的影响")
print(f"   ✓ 熟练使用concurrent.futures进行并发编程")
print(f"   ✓ 能够选择合适的并发策略")

### 同步机制与通信 [⭐⭐进阶]
**知识点说明**：同步机制是并发编程中的关键技术，用于协调多个线程或进程的执行顺序，避免竞态条件和数据冲突。掌握同步机制对于构建稳定可靠的并发程序非常重要。

**学习要求**：
- 掌握常见的同步原语（Lock、Event、Condition等）
- 理解线程间和进程间的通信机制
- 能够使用队列进行数据传递
- 掌握生产者-消费者模式的实现

**案例要求**：
- 实现完整的同步机制示例
- 进行线程安全和进程安全练习
- 应用同步机制解决实际问题
- 验证点：能独立构建线程安全的并发程序

In [ ]:
print("🔐 同步机制与通信:")
print("=" * 50)

import threading
import multiprocessing
import concurrent.futures
import time
import queue
from typing import List, Dict, Any, Callable, Optional
from dataclasses import dataclass
from enum import Enum
import random
import json
from datetime import datetime

# 1. 线程同步机制
print(f"📝 1. 线程同步机制:")

# 1.1 Lock（锁）
print(f"\n   🔒 1.1 Lock（锁）:")

class ThreadSafeCounter:
    """线程安全的计数器"""
    
    def __init__(self):
        self.count = 0
        self.lock = threading.Lock()
    
    def increment(self):
        """线程安全的递增"""
        with self.lock:
            current = self.count
            time.sleep(0.001)  # 模拟处理时间
            self.count = current + 1
    
    def get_count(self):
        """获取当前计数"""
        with self.lock:
            return self.count

def demonstrate_lock():
    """演示锁的使用"""
    counter = ThreadSafeCounter()
    threads = []
    
    def worker():
        for _ in range(100):
            counter.increment()
    
    print(f"   创建10个线程，每个线程递增100次")
    
    # 创建并启动线程
    for i in range(10):
        thread = threading.Thread(target=worker)
        threads.append(thread)
        thread.start()
    
    # 等待所有线程完成
    for thread in threads:
        thread.join()
    
    final_count = counter.get_count()
    print(f"   最终计数: {final_count} (期望: 1000)")
    print(f"   {'✅ 线程安全' if final_count == 1000 else '❌ 存在竞态条件'}")

demonstrate_lock()

# 1.2 Event（事件）
print(f"\n   📢 1.2 Event（事件）:")

def demonstrate_event():
    """演示事件的使用"""
    event = threading.Event()
    
    def worker(name: str):
        print(f"   工作线程{name}: 等待事件...")
        event.wait()  # 等待事件被设置
        print(f"   工作线程{name}: 收到事件，开始工作")
    
    def signaler():
        time.sleep(0.1)
        print(f"   信号线程: 设置事件")
        event.set()  # 设置事件
    
    print(f"   事件机制演示:")
    
    # 创建工作线程
    workers = []
    for i in range(3):
        thread = threading.Thread(target=worker, args=(f"{i+1}",))
        workers.append(thread)
        thread.start()
    
    # 创建信号线程
    signal_thread = threading.Thread(target=signaler)
    signal_thread.start()
    
    # 等待所有线程完成
    for thread in workers:
        thread.join()
    signal_thread.join()
    
    print(f"   所有线程完成")

demonstrate_event()

# 1.3 Condition（条件变量）
print(f"\n   ⚖️ 1.3 Condition（条件变量）:")

class BoundedBuffer:
    """有界缓冲区（生产者-消费者模式）"""
    
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.buffer = []
        self.condition = threading.Condition()
    
    def produce(self, item: Any):
        """生产物品"""
        with self.condition:
            while len(self.buffer) >= self.capacity:
                print(f"   缓冲区已满，生产者等待")
                self.condition.wait()
            
            self.buffer.append(item)
            print(f"   生产: {item}, 缓冲区大小: {len(self.buffer)}")
            self.condition.notify_all()
    
    def consume(self) -> Any:
        """消费物品"""
        with self.condition:
            while len(self.buffer) == 0:
                print(f"   缓冲区为空，消费者等待")
                self.condition.wait()
            
            item = self.buffer.pop(0)
            print(f"   消费: {item}, 缓冲区大小: {len(self.buffer)}")
            self.condition.notify_all()
            return item

def demonstrate_condition():
    """演示条件变量的使用"""
    buffer = BoundedBuffer(capacity=3)
    
    def producer():
        for i in range(5):
            buffer.produce(f"物品{i+1}")
            time.sleep(0.05)
    
    def consumer():
        for i in range(5):
            item = buffer.consume()
            time.sleep(0.08)
    
    print(f"   生产者-消费者模式演示:")
    
    # 创建生产者和消费者线程
    producer_thread = threading.Thread(target=producer)
    consumer_thread = threading.Thread(target=consumer)
    
    producer_thread.start()
    consumer_thread.start()
    
    producer_thread.join()
    consumer_thread.join()
    
    print(f"   生产者-消费者演示完成")

demonstrate_condition()

# 2. 线程间通信
print(f"\n📝 2. 线程间通信:")

# 2.1 Queue（队列）
print(f"\n   📬 2.1 Queue（队列）:")

def demonstrate_queue():
    """演示队列的使用"""
    data_queue = queue.Queue(maxsize=5)
    result_queue = queue.Queue()
    
    def producer():
        """生产者线程"""
        for i in range(8):
            try:
                data_queue.put(f"数据{i+1}", timeout=0.1)
                print(f"   生产: 数据{i+1}")
                time.sleep(0.02)
            except queue.Full:
                print(f"   队列已满，跳过数据{i+1}")
    
    def consumer():
        """消费者线程"""
        while True:
            try:
                data = data_queue.get(timeout=0.1)
                processed = f"处理_{data}"
                result_queue.put(processed)
                print(f"   消费: {data} -> {processed}")
                data_queue.task_done()
                time.sleep(0.03)
            except queue.Empty:
                break
    
    print(f"   线程间队列通信演示:")
    
    # 创建生产者和消费者线程
    producer_thread = threading.Thread(target=producer)
    consumer_thread = threading.Thread(target=consumer)
    
    producer_thread.start()
    time.sleep(0.05)  # 让生产者先运行
    consumer_thread.start()
    
    producer_thread.join()
    consumer_thread.join()
    
    # 收集结果
    results = []
    while not result_queue.empty():
        results.append(result_queue.get())
    
    print(f"\n   处理结果: {results}")
    print(f"   结果数量: {len(results)}")

demonstrate_queue()

# 3. 进程间通信
print(f"\n📝 3. 进程间通信:")

# 3.1 multiprocessing.Queue
print(f"\n   🔄 3.1 multiprocessing.Queue:")

def demonstrate_process_queue():
    """演示进程间队列通信"""
    def worker(input_queue: multiprocessing.Queue, output_queue: multiprocessing.Queue):
        """工作进程"""
        while True:
            try:
                data = input_queue.get(timeout=1)
                if data == "STOP":
                    break
                
                # 处理数据
                processed = f"进程处理_{data}"
                output_queue.put(processed)
                time.sleep(0.1)
            except queue.Empty:
                break
    
    print(f"   进程间队列通信演示:")
    
    # 创建进程间队列
    input_queue = multiprocessing.Queue()
    output_queue = multiprocessing.Queue()
    
    # 创建工作进程
    processes = []
    for i in range(2):
        process = multiprocessing.Process(
            target=worker,
            args=(input_queue, output_queue)
        )
        processes.append(process)
        process.start()
    
    # 发送任务
    tasks = ["任务1", "任务2", "任务3", "任务4"]
    for task in tasks:
        input_queue.put(task)
        print(f"   发送任务: {task}")
    
    # 发送停止信号
    for _ in range(2):
        input_queue.put("STOP")
    
    # 等待进程完成
    for process in processes:
        process.join()
    
    # 收集结果
    results = []
    while not output_queue.empty():
        results.append(output_queue.get())
    
    print(f"\n   处理结果: {results}")
    print(f"   结果数量: {len(results)}")

# 注意：在Jupyter环境中，multiprocessing可能会有问题
# 这里只演示代码结构，不实际执行
print(f"   进程间通信代码结构已准备")
print(f"   # demonstrate_process_queue()  # 在独立Python文件中运行")

# 3.2 共享内存
print(f"\n   💾 3.2 共享内存:")

def demonstrate_shared_memory():
    """演示共享内存的使用"""
    print(f"   共享内存类型:")
    print(f"   - multiprocessing.Value: 单个值共享")
    print(f"   - multiprocessing.Array: 数组共享")
    print(f"   - multiprocessing.Manager: 复杂对象共享")
    
    # 代码示例（不实际执行）
    example_code = '''
# 创建共享变量
counter = multiprocessing.Value('i', 0)  # 整数
data_array = multiprocessing.Array('d', [0.0] * 10)  # 浮点数组

# 使用Manager共享复杂对象
with multiprocessing.Manager() as manager:
    shared_dict = manager.dict()
    shared_list = manager.list()
'''
    
    print(f"\n   共享内存示例代码:")
    print(example_code)

demonstrate_shared_memory()

print(f"\n✅ 同步机制与通信完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握常见的同步原语（Lock、Event、Condition等）")
print(f"   ✓ 理解线程间和进程间的通信机制")
print(f"   ✓ 能够使用队列进行数据传递")
print(f"   ✓ 掌握生产者-消费者模式的实现")

### 实际应用案例 [⭐⭐进阶]
**知识点说明**：实际应用案例是将并发编程知识转化为实用技能的关键环节。通过解决真实世界的问题，学生能够深入理解并发编程的价值和应用场景。

**学习要求**：
- 掌握并发下载和数据处理技术
- 理解Web爬虫的并发实现
- 能够构建高效的批量处理系统
- 掌握性能优化和调试技巧

**案例要求**：
- 实现完整的并发应用系统
- 进行性能对比和优化练习
- 应用并发编程解决实际问题
- 验证点：能独立构建高性能的并发应用

In [ ]:
print("🚀 实际应用案例:")
print("=" * 50)

import threading
import multiprocessing
import concurrent.futures
import time
import queue
from typing import List, Dict, Any, Callable, Optional
from dataclasses import dataclass
from enum import Enum
import random
import json
from datetime import datetime
import urllib.request
import urllib.parse
import hashlib

# 1. 并发下载器
print(f"📝 1. 并发下载器:")

@dataclass
class DownloadTask:
    """下载任务"""
    url: str
    filename: str
    timeout: int = 10

class ConcurrentDownloader:
    """并发下载器"""
    
    def __init__(self, max_workers: int = 5):
        self.max_workers = max_workers
        self.results = []
        self.lock = threading.Lock()
    
    def download_file(self, task: DownloadTask) -> Dict[str, Any]:
        """下载单个文件"""
        start_time = time.time()
        
        try:
            # 模拟下载（实际使用urllib或requests）
            time.sleep(random.uniform(0.1, 0.5))  # 模拟网络延迟
            
            # 模拟成功下载
            file_size = random.randint(1024, 10240)
            elapsed_time = time.time() - start_time
            
            result = {
                "url": task.url,
                "filename": task.filename,
                "status": "success",
                "file_size": file_size,
                "elapsed_time": elapsed_time,
                "download_speed": file_size / elapsed_time if elapsed_time > 0 else 0
            }
            
            # 线程安全地添加结果
            with self.lock:
                self.results.append(result)
            
            return result
            
        except Exception as e:
            error_result = {
                "url": task.url,
                "filename": task.filename,
                "status": "error",
                "error": str(e),
                "elapsed_time": time.time() - start_time
            }
            
            with self.lock:
                self.results.append(error_result)
            
            return error_result
    
    def download_files(self, tasks: List[DownloadTask]) -> List[Dict[str, Any]]:
        """并发下载多个文件"""
        print(f"   开始下载{len(tasks)}个文件，最大并发数: {self.max_workers}")
        
        start_time = time.time()
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            # 提交所有下载任务
            future_to_task = {executor.submit(self.download_file, task): task for task in tasks}
            
            # 等待所有任务完成
            completed = 0
            for future in concurrent.futures.as_completed(future_to_task):
                completed += 1
                result = future.result()
                status = "✅" if result["status"] == "success" else "❌"
                print(f"   进度: {completed}/{len(tasks)} {status} {result['filename']}")
        
        total_time = time.time() - start_time
        
        # 统计结果
        successful = sum(1 for r in self.results if r["status"] == "success")
        failed = len(self.results) - successful
        total_size = sum(r.get("file_size", 0) for r in self.results if r["status"] == "success")
        
        print(f"\n   下载完成统计:")
        print(f"   总时间: {total_time:.2f}秒")
        print(f"   成功: {successful}, 失败: {failed}")
        print(f"   总下载量: {total_size} 字节")
        print(f"   平均速度: {total_size/total_time:.0f} 字节/秒")
        
        return self.results

def demonstrate_concurrent_downloader():
    """演示并发下载器"""
    # 创建下载任务
    tasks = [
        DownloadTask("https://example.com/file1.txt", "file1.txt"),
        DownloadTask("https://example.com/file2.txt", "file2.txt"),
        DownloadTask("https://example.com/file3.txt", "file3.txt"),
        DownloadTask("https://example.com/file4.txt", "file4.txt"),
        DownloadTask("https://example.com/file5.txt", "file5.txt"),
        DownloadTask("https://example.com/file6.txt", "file6.txt"),
    ]
    
    # 创建下载器
    downloader = ConcurrentDownloader(max_workers=3)
    
    # 执行下载
    results = downloader.download_files(tasks)
    
    # 显示详细结果
    print(f"\n   详细结果:")
    for result in results:
        if result["status"] == "success":
            print(f"   ✅ {result['filename']}: {result['file_size']}字节, "
                  f"{result['elapsed_time']:.2f}秒")
        else:
            print(f"   ❌ {result['filename']}: {result['error']}")

demonstrate_concurrent_downloader()

# 2. 并发数据处理
print(f"\n📝 2. 并发数据处理:")

class DataProcessor:
    """并发数据处理器"""
    
    def __init__(self, max_workers: int = 4):
        self.max_workers = max_workers
    
    def process_data_item(self, data_item: Dict[str, Any]) -> Dict[str, Any]:
        """处理单个数据项"""
        # 模拟数据处理
        time.sleep(random.uniform(0.05, 0.15))
        
        # 数据转换
        processed = {
            "id": data_item["id"],
            "original_value": data_item["value"],
            "processed_value": data_item["value"] * 2,
            "hash": hashlib.md5(str(data_item["value"]).encode()).hexdigest()[:8],
            "timestamp": datetime.now().isoformat()
        }
        
        return processed
    
    def process_batch(self, data_list: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """批量处理数据"""
        print(f"   开始处理{len(data_list)}条数据，最大并发数: {self.max_workers}")
        
        start_time = time.time()
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            # 使用map函数批量处理
            results = list(executor.map(self.process_data_item, data_list))
        
        elapsed_time = time.time() - start_time
        
        print(f"   处理完成，耗时: {elapsed_time:.2f}秒")
        print(f"   处理速度: {len(results)/elapsed_time:.1f} 条/秒")
        
        return results
    
    def compare_performance(self, data_list: List[Dict[str, Any]]) -> Dict[str, float]:
        """对比串行和并发性能"""
        print(f"\n   性能对比测试:")
        
        # 串行处理
        start_time = time.time()
        serial_results = [self.process_data_item(item) for item in data_list]
        serial_time = time.time() - start_time
        
        # 并发处理
        start_time = time.time()
        concurrent_results = self.process_batch(data_list)
        concurrent_time = time.time() - start_time
        
        # 计算性能提升
        speedup = serial_time / concurrent_time if concurrent_time > 0 else 0
        
        print(f"   串行处理: {serial_time:.2f}秒")
        print(f"   并发处理: {concurrent_time:.2f}秒")
        print(f"   性能提升: {speedup:.2f}x")
        
        return {
            "serial_time": serial_time,
            "concurrent_time": concurrent_time,
            "speedup": speedup
        }

def demonstrate_data_processor():
    """演示并发数据处理器"""
    # 生成测试数据
    test_data = [
        {"id": i, "value": random.randint(1, 1000)}
        for i in range(20)
    ]
    
    print(f"   测试数据示例: {test_data[:3]}...")
    
    # 创建处理器
    processor = DataProcessor(max_workers=4)
    
    # 处理数据
    results = processor.process_batch(test_data)
    
    # 显示部分结果
    print(f"\n   处理结果示例:")
    for result in results[:3]:
        print(f"   {result}")
    
    # 性能对比
    performance = processor.compare_performance(test_data[:10])

demonstrate_data_processor()

# 3. Web爬虫并发实现
print(f"\n📝 3. Web爬虫并发实现:")

@dataclass
class WebPage:
    """网页数据结构"""
    url: str
    title: str = ""
    content: str = ""
    links: List[str] = None
    status: str = "pending"
    error: str = ""
    
    def __post_init__(self):
        if self.links is None:
            self.links = []

class ConcurrentWebCrawler:
    """并发网页爬虫"""
    
    def __init__(self, max_workers: int = 5, delay: float = 0.1):
        self.max_workers = max_workers
        self.delay = delay
        self.visited_urls = set()
        self.lock = threading.Lock()
    
    def fetch_page(self, url: str) -> WebPage:
        """获取单个网页"""
        # 检查是否已访问
        with self.lock:
            if url in self.visited_urls:
                return WebPage(url, status="skipped")
            self.visited_urls.add(url)
        
        page = WebPage(url)
        
        try:
            # 模拟网络请求
            time.sleep(random.uniform(0.05, 0.2))
            
            # 模拟网页内容
            page.title = f"页面标题_{url.split('/')[-1] or 'home'}"
            page.content = f"这是{url}的内容..."
            page.links = [f"{url}/link{i}" for i in range(1, 4)]
            page.status = "success"
            
        except Exception as e:
            page.status = "error"
            page.error = str(e)
        
        # 请求延迟
        time.sleep(self.delay)
        
        return page
    
    def crawl_urls(self, urls: List[str]) -> List[WebPage]:
        """并发爬取多个URL"""
        print(f"   开始爬取{len(urls)}个URL，最大并发数: {self.max_workers}")
        
        start_time = time.time()
        pages = []
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            # 提交所有爬取任务
            future_to_url = {executor.submit(self.fetch_page, url): url for url in urls}
            
            # 收集结果
            completed = 0
            for future in concurrent.futures.as_completed(future_to_url):
                completed += 1
                page = future.result()
                pages.append(page)
                
                status_icon = {
                    "success": "✅",
                    "error": "❌",
                    "skipped": "⏭️"
                }.get(page.status, "❓")
                
                print(f"   进度: {completed}/{len(urls)} {status_icon} {page.url}")
        
        elapsed_time = time.time() - start_time
        
        # 统计结果
        successful = sum(1 for p in pages if p.status == "success")
        failed = sum(1 for p in pages if p.status == "error")
        skipped = sum(1 for p in pages if p.status == "skipped")
        
        print(f"\n   爬取完成统计:")
        print(f"   总时间: {elapsed_time:.2f}秒")
        print(f"   成功: {successful}, 失败: {failed}, 跳过: {skipped}")
        print(f"   平均速度: {successful/elapsed_time:.1f} 页/秒")
        
        return pages

def demonstrate_web_crawler():
    """演示并发网页爬虫"""
    # 创建测试URL列表
    urls = [
        "https://example.com",
        "https://example.com/about",
        "https://example.com/products",
        "https://example.com/contact",
        "https://example.com/blog",
        "https://example.com/news",
        "https://example.com/services",
        "https://example.com",
    ]  # 重复URL测试去重
    
    # 创建爬虫
    crawler = ConcurrentWebCrawler(max_workers=3, delay=0.05)
    
    # 执行爬取
    pages = crawler.crawl_urls(urls)
    
    # 显示部分结果
    print(f"\n   爬取结果示例:")
    for page in pages[:3]:
        if page.status == "success":
            print(f"   ✅ {page.url}: {page.title}, 链接数: {len(page.links)}")
        elif page.status == "error":
            print(f"   ❌ {page.url}: {page.error}")
        else:
            print(f"   ⏭️ {page.url}: 已跳过")

demonstrate_web_crawler()

print(f"\n✅ 实际应用案例完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握并发下载和数据处理技术")
print(f"   ✓ 理解Web爬虫的并发实现")
print(f"   ✓ 能够构建高效的批量处理系统")
print(f"   ✓ 掌握性能优化和调试技巧")

## 📋 学习总结

### ✅ 知识清单达成情况验证

**4.6 多线程与多进程 [⭐⭐进阶]**
- ✅ 掌握线程和进程的基本概念和区别
- ✅ 熟练使用concurrent.futures进行并发编程
- ✅ 理解GIL对Python并发性能的影响
- ✅ 能够选择合适的并发策略
- ✅ 掌握常见的同步原语（Lock、Event、Condition等）
- ✅ 理解线程间和进程间的通信机制
- ✅ 能够使用队列进行数据传递
- ✅ 掌握生产者-消费者模式的实现
- ✅ 能独立构建高效的并发程序

### 🎯 与LangChain学习的关联

**并发处理重要性**：
- 并发处理是LangChain批量API调用的基础
- 线程池模式影响LangChain的并行处理能力
- I/O密集型任务优化对LangChain性能至关重要
- 生产者-消费者模式在LangChain流式处理中应用广泛

**实际应用场景**：
- LangChain的并行LLM调用需要线程池支持
- 批量文档处理和向量化操作依赖并发编程
- API工具的并发执行提升响应速度
- 流式响应和回调系统使用类似的同步机制

### 📚 进阶学习建议

1. **练习建议**：
   - 多练习I/O密集型任务的并发优化
   - 深入学习异步编程（asyncio）
   - 掌握分布式并发和消息队列

2. **扩展学习**：
   - 学习asyncio和aiohttp异步框架
   - 了解Celery分布式任务队列
   - 探索协程和生成器的并发应用

3. **实际应用**：
   - 构建高并发的Web服务
   - 开发分布式数据处理系统
   - 实现实时数据流处理平台

### 🔧 常见错误与注意事项

1. **滥用多线程**：
   ```python
   # 错误：对CPU密集型任务使用多线程
   def cpu_intensive_task():
       # 大量计算，GIL会阻止真正的并行
       result = sum(i * i for i in range(1000000))
       return result
   
   # 正确：对CPU密集型任务使用多进程
   with ProcessPoolExecutor() as executor:
       result = executor.submit(cpu_intensive_task)
   ```

2. **忽略线程安全**：
   ```python
   # 错误：多线程共享数据没有同步
   counter = 0
   def increment():
       global counter
       counter += 1  # 竞态条件
   
   # 正确：使用锁保护共享数据
   counter = 0
   lock = threading.Lock()
   def increment():
       global counter
       with lock:
           counter += 1
   ```

3. **过度创建线程**：
   ```python
   # 错误：为每个任务创建新线程
   for task in tasks:
       thread = threading.Thread(target=process_task, args=(task,))
       thread.start()  # 可能创建数千个线程
   
   # 正确：使用线程池控制并发数
   with ThreadPoolExecutor(max_workers=10) as executor:
       futures = [executor.submit(process_task, task) for task in tasks]
   ```

4. **死锁问题**：
   ```python
   # 错误：可能导致死锁
   lock1 = threading.Lock()
   lock2 = threading.Lock()
   
   def thread1():
       with lock1:
           with lock2:  # 可能死锁
               pass
   
   def thread2():
       with lock2:
           with lock1:  # 可能死锁
               pass
   
   # 正确：按固定顺序获取锁
   def thread1():
       with lock1:
           with lock2:
               pass
   
   def thread2():
       with lock1:  # 相同顺序
           with lock2:
               pass
   ```

5. **忽略异常处理**：
   ```python
   # 错误：忽略线程中的异常
   def worker():
       # 可能抛出异常，但主线程无法捕获
       risky_operation()
   
   # 正确：在worker函数中处理异常
   def worker():
       try:
           risky_operation()
       except Exception as e:
           logger.error(f"Worker error: {e}")
   
   # 或者使用future.result()获取异常
   future = executor.submit(worker)
   try:
       result = future.result()
   except Exception as e:
       print(f"Task failed: {e}")
   ```

6. **资源泄漏**：
   ```python
   # 错误：没有正确关闭资源
   executor = ThreadPoolExecutor(max_workers=10)
   # 使用executor但没有shutdown
   
   # 正确：使用上下文管理器
   with ThreadPoolExecutor(max_workers=10) as executor:
       # 使用executor
       pass  # 自动shutdown
   
   # 或者手动shutdown
   executor = ThreadPoolExecutor(max_workers=10)
   try:
       # 使用executor
       pass
   finally:
       executor.shutdown(wait=True)
   ```

### 🌐 性能优化建议

**并发策略选择**：
- **I/O密集型任务**：优先使用ThreadPoolExecutor
- **CPU密集型任务**：使用ProcessPoolExecutor
- **混合型任务**：结合使用线程池和进程池
- **简单任务**：使用map函数而非submit

**性能调优**：
- 合理设置max_workers（通常为CPU核心数的2-4倍）
- 使用队列控制任务流量，避免内存溢出
- 实现优雅关闭，确保资源正确释放
- 监控线程/进程数量，防止资源耗尽

---

**🎉 恭喜完成多线程与多进程学习！**

你已经掌握了Python并发编程的核心技能，能够熟练使用concurrent.futures进行线程和进程管理，理解了GIL的影响，掌握了同步机制和通信技术，为后续学习LangChain的并行处理和异步编程奠定了坚实基础。

## 🚀 下一步学习预告

**继续第四节：常用标准库**：
- 4.7 数据序列化与存储 [⭐⭐进阶]
- 4.8 系统信息获取 [⭐⭐进阶]
- 4.9 常用工具模块 [⭐⭐进阶]

继续加油，完成Python标准库的学习！